In [ ]:
# ✅todo : 1. colab 용으로 requirements.txt을 하나 더 만든다.
# ✅ todo : 1. turnonselenium 내용을 colab용으로 추가한다.
# ✅todo : 1. blogger를 모듈화해서 사용하기 쉽게 만든다.
# ✅todo : 1. 쓰레딩 시키는 것도 함수화한다.
# ✅todo : 1. 코랩은 쓰레딩 몇개를 시키는게 적당한지를 확인한다 = 2개 (2코어라서)
# ✅todo : 1. 코랩크롤링 시에, user-agent 사용하도록 한다.
# todo : duckduckgo 관련자료 서칭 대체품 찾기
# todo : 쉬는 시간 가지도록 만들기
# todo : 만들어진 컨텐츠들 중에 document 없는 경우 찾기


# ✅todo : 1딸깍 테스트
# todo : 1딸깍 반복문에 쉬는 시간 부여하기
# todo : 2딸깍 테스트
# todo : 2딸깍 반복문에 쉬는 시간 부여하기
# todo : 3딸깍 테스트
# todo : 3딸깍 반복문에 쉬는 시간 부여하기

# ✅todo : 1딸깍 테스트 - colab
# todo : 2딸깍 테스트 - colab
# todo : 3딸깍 테스트 - colab

# todo : 추가 키워드 수집 작업 시 기존 csv를 덮어쓸지 말지를 결정하도록 해주세요. - 현재는 덮어씌우지 않는 것으로 되어있음.
# todo : 추가 키워드 수집 작업 시 기존 suitable keywords 리스트를 고려하도록 해주세요. (있으면 재반복해서 수행하지는 않는 것으로.)

# todo : 업로드가 성공적으로 수행되면은 옮기는 것으로 변경하세요.
# todo : 노출순위 따질 때 1) 한번정도는 아래로 스크롤해서 가져오기 2) 네이버 블로그도 순위에 포함시키기.
# todo : 글 생성에 사용한 키워드 저장하고 유사키워드 문지방수준  좀 줄여야 할 것 같아. 너무 관계없는 애들도 같이 묶이는 것 같어.
# todo : 블로그에 올릴 때, 성공한 글에 대해서만 uploaded로 옮기도록 해야할 것 같어.
# todo : suitable check를 할 때, 기존 screened_keywords에 있는 키워드들은 제외하도록 해야할 것 같아. 그러기 위해서는 키워드 수집 시에 collected_keywords에다가 suitability_checked 여부를 표시를 해두어야 할 것 같습니다.


# 기본실행

In [ ]:
# 기본적인 환경설정을 합니다.
isHeadless = True
verbose = True


from module.Crawler import Crawler
from module.File_manager import File_manager
from module.Blogger import Blogger
from module.AIAgent import AIAgent
from module.Uploader import Uploader

# 경고를 무시합니다.
import warnings
warnings.filterwarnings("ignore")

# 환경변수 저장 (.env) 확인
from dotenv import load_dotenv
import os

load_dotenv()


verbose = True
isHeadless = True

# # tester용 객체들을 만듭니다.
# verbose = True
# isHeadless = True
# crawler_tester = Crawler(isHeadless = isHeadless, verbose = verbose)
# file_manager_tester = File_manager(blogname='tester', verbose=verbose)
# aiagent_tester = AIAgent(blogname='tester', verbose = verbose)
# uploader_tester = Uploader(blogname='tester', verbose = verbose)
# blogger_tester = Blogger(blogname='tester', verbose = verbose, isHeadless = isHeadless)

def thread_bloggers(func, bloggers, **kwargs):
    import threading

    threads = []
    for blogger in bloggers.values():
        # args 튜플에 blogger를 포함시키고, kwargs 딕셔너리를 threading.Thread에 전달
        thread = threading.Thread(target=func, args=(blogger,), kwargs=kwargs)
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
def create_partial_bloggers(total_bloggers:dict, selected_bloggers:list) :
  partial_bloggers = {}
  for blogger in selected_bloggers :
    partial_bloggers[blogger] = total_bloggers[blogger]
  return partial_bloggers



# blogger 클래스를 호출합니다.
from module.Blogger import Blogger
blog_names = ['statifi', 'kindmom', 'sweetkiwi', 'takuz', 'morningbbobbo']
bloggers = {}
for blogname in blog_names:
    print(f'🌐 now loading : {blogname}.')
    bloggers[blogname] = Blogger(blogname = blogname, verbose = verbose, isHeadless = isHeadless)




In [10]:
fron = {
    'a' : 'hi',
    'a' : 'bye'
}

In [19]:
for k, v in fron.items() :
    print(k, v)
    print('---')    

a bye
---


# 클릭1 : 키워드 수집

쓰레딩


In [ ]:
# 환경설정

# 깊이 정의
depth = int(input('키워드를 수집할 깊이를 입력하세요. (최대 3 이상은 비추천) :'))

# 단어 정의
recommended = {
    "건강과 웰빙": ["요가", "마인드풀니스", "웰니스 리트릿", "건강한 간식", "명상", "필라테스", "건강한 식단", "건강 추세", "힐링", "건강 검진"],
    "기업과 기업가": ["기업가 정신", "크라우드펀딩", "비즈니스 모델 혁신", "기술 벤처", "벤처 투자", "스타트업 생태계"],
    "재택 근무 및 자기 계발 블로그": ["재택 근무 팁", "자기계발 책 추천", "온라인 학습 플랫폼", "시간 관리 기술", "생산성 향상", "사이드 허슬", "재택 근무 환경", "프리랜서 가이드", "자기 관리", "모티베이션 증진"],
    "건강 및 웰니스 블로그": ["건강한 식단", "정신 건강", "스트레스 관리", "홈 피트니스", "명상 기법", "웰니스 라이프스타일", "영양 정보", "건강 검진 가이드", "체중 관리", "건강한 습관"],
    
}

subjects_n_words_for_statifi = {}
subjects_n_words_for_kindmom = {}
subjects_n_words_for_sweetkiwi = {}
subjects_n_words_for_takuz = {}
subjects_n_words_for_morningbbobbo = {}

# 각 블로거의 주제 및 키워드 데이터

total_subjects_n_words = {
    'statifi': subjects_n_words_for_statifi,
    'kindmom': subjects_n_words_for_kindmom,
    'sweetkiwi': subjects_n_words_for_sweetkiwi,
    'takuz': subjects_n_words_for_takuz,
    'morningbbobbo': subjects_n_words_for_morningbbobbo
}



# 함수정의
def collect_keywords_for_blogger(blogger, total_subjects_n_words, depth, save):
    subjects_n_words = total_subjects_n_words[blogger.blogname]
    blogger.collect_keywords(subjects_n_words=subjects_n_words, depth=depth, save=save)


# 실행
thread_bloggers(
    func=collect_keywords_for_blogger,
    bloggers=bloggers,
    total_subjects_n_words=total_subjects_n_words,
    depth=depth,
    save=True
)

tester 테스트

In [ ]:
# 단일 블로거로 테스트

test_blog = 'tester'
total_subjects_n_words = {test_blog : {'파이어족' : ['파이어족']} }
depth = 1

# 함수정의
def collect_keywords_for_blogger(blogger, total_subjects_n_words, depth, save):
    subjects_n_words = total_subjects_n_words[blogger.blogname]
    blogger.collect_keywords(subjects_n_words=subjects_n_words, depth=depth, save=save)


collect_keywords_for_blogger(blogger_tester, total_subjects_n_words, depth, save=True)

vectorstore_stored_keywords = blogger_tester.keyword_ai.vectorstore_list.index.to_list()
if len(vectorstore_stored_keywords) > 0 :
    print('🌐 키워드 수집이 완료되었습니다.')    
    blogger_tester.keyword_ai.vectorstore_clear()

# 클릭 2 : 포스팅 생성

쓰레딩

In [ ]:
# 환경설정
num_contents_creation = int(input('생성할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요 '))

# 함수정의
def create_contents(blogger, num_contents_creation=5):
    print(f"Starting thread for {blogger.blogname}")
    blogger.create_contents(num_contents_creation)

# 실행
thread_bloggers(create_contents, bloggers, num_contents_creation=num_contents_creation)

tester 테스트코드

In [ ]:
# 단일블로거로 테스트

# 함수정의
def create_contents(blogger, num_contents_creation=5):
    print(f"Starting thread for {blogger.blogname}")
    blogger.create_contents(num_contents_creation)

# 함수실행

blogger_tester.keyword_ai.vectorstore_save_texts('파이어족 특징') # 예시 키워드 저장
num_of_created_contents_now = len(blogger_tester.file_manager.get_file_names())

try :
    create_contents(blogger_tester, num_contents_creation=1)
finally :
    blogger_tester.keyword_ai.vectorstore_clear()

# 실행결과 확인
num_of_created_contents_after = len(blogger_tester.file_manager.get_file_names())
if (num_of_created_contents_after - num_of_created_contents_now) > 0 :
    print('🌐 컨텐츠 생성이 완료되었습니다.')  


# 클릭3 : 티스토리 블로그 업로드

순차반복문

In [ ]:
# 환경설정
num_contents_upload = int(input('업로드 할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요. '))
assert num_contents_upload is not None, '업로드할 컨텐츠 갯수를 입력하세요.'
assert num_contents_upload <= 15 , '15개 이하로 업로드 가능합니다.'

# 함수정의
def upload(blogger, num_contents_upload):
    blogger.upload_contents(num_contents_upload)

for blogname in blog_names :
    upload(bloggers[blogname], num_contents_upload)


쓰레딩

In [ ]:
# 환경설정
num_contents_upload = int(input('업로드 할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요. '))
assert num_contents_upload is not None, '업로드할 컨텐츠 갯수를 입력하세요.'
assert num_contents_upload <= 15 , '15개 이하로 업로드 가능합니다.'

# 함수정의
def upload(blogger, num_contents_upload):
    blogger.upload_contents(num_contents_upload)

# 실행
thread_bloggers(
    func=upload,
    bloggers=bloggers,
    num_contents_upload=num_contents_upload)

테스트코드 (blogger_tester)

In [ ]:
# 함수정의
def upload(blogger, num_contents_upload = 1):
    blogger.upload_contents(num_contents_upload)

# 함수실행

os.environ["tester_NEW_POST_URL"] = str(r"https://statifi.tistory.com/manage/newpost/?type=post&returnURL=%2Fmanage%2Fposts%2F")
os.environ["tester_ID"] = "jsj950611@naver.com"
os.environ["tester_PW"] = "Zhzkzhffk1!"


upload(blogger_tester, num_contents_upload=1)

# 테스트코드는 여기 이하에 작성

In [3]:
# 남은 글의 갯수
for blogname in blog_names:
    print(len(bloggers[blogname].file_manager.get_file_names()))

24
25
0
62
0


In [5]:
# 남은 키워드의 갯수
for blogname in blog_names:
    print(len(bloggers[blogname].keyword_ai.vectorstore_list))

0
630
58
947
45


In [6]:
# 이름순서
for blogname in blog_names:
    print(blogname)

statifi
kindmom
sweetkiwi
takuz
morningbbobbo


In [46]:
df = bloggers['takuz'].file_manager.get_keywords('collected_keywords')
isNaverIn = []
for all_links in df['all_links'] :
    done = False
    for link in all_links :        
        if link is None : 
            isNaverIn.append(False)
            done = True
            break
        if 'naver' in link :
            isNaverIn.append(True)
            done = True
            break
    if done == False :
        isNaverIn.append(False)
    
        

True
True
Data loaded from JSON


In [59]:
df['isNaverIn'] = isNaverIn
len(df[df['isNaverIn'] == True][df['tistory_rank_at_google'] == '노출없음'])

1996

In [62]:
a = [1,2,3,]
b = [2,3,4]




[1]